# QA tests on SkySub (v1.1 by JXP)

## Load 'true' model sky spectrum

In [1]:
# imports
import specter.throughput as spec_thru
import specter.psf.spotgrid as SpotGridPSF
#sys.path.append(os.path.abspath("/Users/xavier/DESI/desisim/py/"))
#sys.path.append(os.path.abspath("/Users/xavier/DESI/desimodel/py/"))
#import interpolation as desi_interp
from desisim import io as desisim_io

In [50]:
# Get truth
simsp2_fil = '/Users/xavier/DESI/TST/20150211/simspec-00000002.fits'
simsp2_hdu = fits.open(simsp2_fil)
simsp2_hdu.info()

Filename: /Users/xavier/DESI/TST/20150211/simspec-00000002.fits
No.    Name         Type      Cards   Dimensions   Format
0    FLUX        PrimaryHDU      13   (31901, 1000)   float32   
1    SKYFLUX     ImageHDU        13   (31901,)     float32   
2    METADATA    BinTableHDU     18   1000R x 4C   [10A, E, J, E]   
3    PHOT_B      ImageHDU        12   (12326, 1000)   float32   
4    SKYPHOT_B   ImageHDU        11   (12326,)     float32   
5    PHOT_R      ImageHDU        12   (11205, 1000)   float32   
6    SKYPHOT_R   ImageHDU        11   (11205,)     float32   
7    PHOT_Z      ImageHDU        12   (12765, 1000)   float32   
8    SKYPHOT_Z   ImageHDU        11   (12765,)     float32   


In [54]:
len(simsp2_hdu)

9

In [5]:
# Wavelength
hdr = simsp2_hdu[0].header
hdr['CRVAL1'], hdr['CDELT1']

(3533.0, 0.1999999999998181)

In [8]:
hdr_skyB = simsp2_hdu[4].header
hdr_skyB['CRVAL1'], hdr_skyB['CDELT1'], hdr_skyB['NAXIS1']

(3533.0, 0.1999999999998181, 12326)

In [9]:
# Generate
waveB = hdr_skyB['CRVAL1'] + (hdr_skyB['CDELT1'])*np.arange(hdr_skyB['NAXIS1'])
skyB = simsp2_hdu[4].data

In [11]:
#Plot
plt.clf()
plt.plot(waveB,skyB)
plt.show()

## Now fiddle about in desispec.sky

In [2]:
#imports
from desispec.io import frame as desi_io_frame
from desispec import sky as dspec_sky
from desispec.io import fibermap as desi_io_fmap

### Added make_qa method (not part of Class yet)

In [6]:
path = '/Users/xavier/DESI/TST/'#20150211/'

In [10]:
# Load
obs_frame = desi_io_frame.read_frame(path+'dogwood/exposures/20150211/00000002/frame-b0-00000002.fits')
fiber_map = desi_io_fmap.read_fibermap(path+'data/20150211/fibermap-00000002.fits')

In [11]:
# Process + checking
reload(dspec_sky)
# Not flat-fielding first..
skymodel, skyflux, skyvar = dspec_sky.compute_sky(obs_frame,fiber_map)

INFO:sky.py:41:compute_sky: starting


INFO:DESI:starting


IndexError: index 526 is out of bounds for axis 0 with size 500

In [31]:
# Resample truth onto extracted spectrum
dw = 0.6
ww = np.arange(3579., 5938.8, dw)
true_flux = desi_interp.resample_flux(ww, waveB, skyB)

In [32]:
#Scale
scl = np.median(skyflux/true_flux)

In [33]:
# Quick comparison
xdb.xplot(skymodel.wave,skyflux,xtwo=ww,ytwo=true_flux*scl)

## QA on Mean model

In [47]:
reload(dspec_sky)
dspec_sky.make_model_qa(skymodel.wave, skyflux, skyvar, waveB, skyB)

scale = 4.73442


In [74]:
#xdb.xplot(fiber_flat.wave, fiber_flat.meanspec, np.sqrt(ms_var))

## $\chi^2$ Test on Sky Fibers